## Pandas (Hierarchical Indexing)

Las Series y DataFrames están orientados principalmente a datos uni y bidimensionales.  

A menudo es útil ir un poco más allá y trabajar con más dimensiones. Para ello Pandas dispone de unos objetos específicos, como son  Panel u Panel4D para manejar datasets de 3 y 4 dimensiones respectivamente. 

Aún así, un patrón común para tratar con más de dos dimensiones es a través del uso de índices jerárquicos, también conocido como Multi-Indexing, que permite incorporar varios niveles en el índice. De esta forma, datos con varias dimensiones pueden ser representados a través de los familiares Series y DataFrames.


In [1]:
import numpy as np
import pandas as pd 

In [2]:
pd.__version__ #versión de pandas

'0.23.4'

### A Multiply Indexed Series

#### The bad way

In [3]:
# Creamos un lista de tuplas, con el estado y el año
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]


In [4]:
# Creamos un array de población
populations = [34343223, 45434534,
              328923,548954,
              2323647, 8984398]

In [5]:
# Creamos una serie de la población, utilizando como índice la lista
# de tuplas
pop = pd.Series(populations, index=index)

In [6]:
# Con este indice rudimentario pordemos utilizar slicing o recorrer
# la serie y extraer la información
pop

(California, 2000)    34343223
(California, 2010)    45434534
(New York, 2000)        328923
(New York, 2010)        548954
(Texas, 2000)          2323647
(Texas, 2010)          8984398
dtype: int64

In [7]:
# Slicing
pop[('California', 2010): ('Texas', 2010)]

(California, 2010)    45434534
(New York, 2000)        328923
(New York, 2010)        548954
(Texas, 2000)          2323647
(Texas, 2010)          8984398
dtype: int64

In [8]:
# Filtering
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    45434534
(New York, 2010)        548954
(Texas, 2010)          8984398
dtype: int64

#### The best way - Pandas MultiIndex

In [9]:
# Podemos crear un múltindice a través de la tupla anterior
index = pd.MultiIndex.from_tuples(index)

Un multindice contiene distintos niveles de indexación y etiquetas para 
cada uno de los valores

In [10]:
# Un multindice contiene distintos niveles de indexación
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [11]:
# reindexamos la serie, utilizando para ello el multiindice
pop = pop.reindex(index)

In [12]:
pop

California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

Las dos primeras columnas son el índice (estado y año), y la tercera son los datos de la serie

In [13]:
# filtramos a través de los dos índices
pop.loc[:, 2010]

California    45434534
New York        548954
Texas          8984398
dtype: int64

Este índice es mucho más conveniente y eficiente

#### MultiIndex as extra dimension

Podríamos haber almacenado la información anterior en un DataFrame con un índice y columnas. De hecho Pandas esta construido con esta idea en mente, de forma que utilizando el método unstack() sobre una Serie multiindexada, devuelve un DataFrame convencional

In [14]:
pop_df = pop.unstack()

In [15]:
pop_df

,2000,2010
California,34343223,45434534
New York,328923,548954
Texas,2323647,8984398


Existe la operación contraria, que nos permite pasar a un multiindice

In [16]:
pop2 = pop_df.stack()

In [17]:
pop2

California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [18]:
type(pop2)

pandas.core.series.Series

In [19]:
pop_df = pop2.unstack()

In [20]:
pop_df

,2000,2010
California,34343223,45434534
New York,328923,548954
Texas,2323647,8984398


In [21]:
pop_df.stack()

California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [22]:
pop_df

,2000,2010
California,34343223,45434534
New York,328923,548954
Texas,2323647,8984398


Visto esto, podrías pensar para que queremos utilizar el índice jerárquico. Pues bien, la razón es simple, el uso de estos multiindices nos permiten representar datos bidimensionales en una Serie y más de dos dimensiones en un DataFrame

En el ejemplo, podemos añadir una nueva columna de datos demográficos al DataFrame

In [23]:
pop

California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [25]:
pop.index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

In [26]:
pop_df = pd.DataFrame({'total': pop, 
                      'under18': [323213, 4839489,
                                 28938, 328329,
                                 23289, 89898]})

In [27]:
pop_df

total  under18
California 2000  34343223   323213
           2010  45434534  4839489
New York   2000    328923    28938
           2010    548954   328329
Texas      2000   2323647    23289
           2010   8984398    89898

In [28]:
pop_df.unstack()

total           under18         
                2000      2010    2000     2010
California  34343223  45434534  323213  4839489
New York      328923    548954   28938   328329
Texas        2323647   8984398   23289    89898

In [29]:
pop_df.stack()

California  2000  total      34343223
                  under18      323213
            2010  total      45434534
                  under18     4839489
New York    2000  total        328923
                  under18       28938
            2010  total        548954
                  under18      328329
Texas       2000  total       2323647
                  under18       23289
            2010  total       8984398
                  under18       89898
dtype: int64

In [30]:
f_u18 = pop_df['under18'] / pop_df['total']

In [31]:
f_u18

California  2000    0.009411
            2010    0.106516
New York    2000    0.087978
            2010    0.598099
Texas       2000    0.010023
            2010    0.010006
dtype: float64

In [32]:
f_u18.unstack()

,2000,2010
California,0.009411,0.106516
New York,0.087978,0.598099
Texas,0.010023,0.010006


### Methods of MultiIndex Creation

La forma más sencilla de crear un Series y DataFrame multiindexados es pasar como índice al crear el objeto una lista con dos o más arrays al constructor

In [33]:
df = pd.DataFrame(np.random.rand(4,2),
                 index = [['a','a','b','b'], ['1','2','1','2']],
                 columns = ['data1', 'data2'])

In [34]:
df

data1     data2
a 1  0.711677  0.503322
  2  0.889695  0.775052
b 1  0.925304  0.182727
  2  0.797534  0.416712

Podemos crear una serie a partir de un diccionario cuyas claves son tuplas

In [35]:
index = {('California', 2000): 32892, 
          ('California', 2010): 289323,
          ('New York', 2000): 32839, 
          ('New York', 2010): 37372,
          ('Texas', 2000): 328938,
          ('Texas', 2010):898989}

In [36]:
pd.Series(index)

California  2000     32892
            2010    289323
New York    2000     32839
            2010     37372
Texas       2000    328938
            2010    898989
dtype: int64

#### Explicit MultiIndex constructors

A partir de pd.MultiIndex podemos construir un multiindice de diversas formas. Este objeto creado se puede pasar en el argumento index al crear una Serie o DataFrame o al reindexarlos (reindex()).

In [37]:
# A partir de una lista de arrays
pd.MultiIndex.from_arrays([['a','a','b','b'], ['1','2','1','2']])

MultiIndex(levels=[['a', 'b'], ['1', '2']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [38]:
# A partir de una lista de tuplas
pd.MultiIndex.from_tuples([('a', 1),('a', 2), ('b', 1),('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [39]:
# A partir del producto cartesiano de dos indices simples
pd.MultiIndex.from_product([['a','b'], ['1','2']])

MultiIndex(levels=[['a', 'b'], ['1', '2']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [40]:
# Directamente sobre el método utilizando los niveles y las etiquetas
pd.MultiIndex(levels=[['a', 'b'], ['1', '2']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], ['1', '2']],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

####  MultiIndex level names

Podemos asignar nombres a los índices a través de una lista o pasando el argumento names a cada uno de los constructores de pd.MultiIndex.

Es útil cuando tenemos varios índices, para mantener la trazabilidad del significado de los índices.

In [41]:
pop.index.names = ['state', 'year']

In [42]:
pop

state       year
California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [43]:
m = pd.MultiIndex.from_arrays([['a','a','b','b'], ['1','2','1','2']],
                         names= ['Vocals', 'Number'])

In [44]:
np.random.rand(4)

array([0.04468288, 0.88113725, 0.62314741, 0.41701608])

In [45]:
pd.Series(np.random.rand(4), index=m)

Vocals  Number
a       1         0.714388
        2         0.135865
b       1         0.972166
        2         0.450627
dtype: float64

In [46]:
pd.DataFrame(np.random.rand(4,2), index=m, columns=['data1', 'data2'])

data1     data2
Vocals Number                    
a      1       0.820870  0.744638
       2       0.877880  0.023545
b      1       0.027854  0.581139
       2       0.192728  0.037732

####  MultiIndex for columns

En un DataFrame, las filas y columnas son completamente simétricas, por lo que es posible también disponer de multiindices a nivel de las columnas

In [47]:
# creamos un indice de años y visitas
index = pd.MultiIndex.from_product([[2013,2014], [1,2]],
                         names= ['year', 'visit'])

In [48]:
# creamos un indice de columnas para personas y tipo
columns = pd.MultiIndex.from_product([['Bob','Guido', 'Sue'], ['HR','Temp']],
                         names= ['subject', 'type'])

In [49]:
# creamos unos datos al azar
data = np.round(np.random.randn(4,6), 1)

In [50]:
data

array([[ 0.2,  0.9, -1.2, -0.5,  0.9, -0.8],
       [-0.2,  0.3, -1. , -1.5, -0.1,  0.5],
       [-0.4, -1.5,  0.4, -0.1,  0.5, -0.3],
       [ 0.1, -0.7, -2.3, -1.8, -1.5,  0.1]])

In [51]:
data[:,::2] *= 10

In [52]:
data

array([[  2. ,   0.9, -12. ,  -0.5,   9. ,  -0.8],
       [ -2. ,   0.3, -10. ,  -1.5,  -1. ,   0.5],
       [ -4. ,  -1.5,   4. ,  -0.1,   5. ,  -0.3],
       [  1. ,  -0.7, -23. ,  -1.8, -15. ,   0.1]])

In [53]:
data += 37

In [54]:
data

array([[39. , 37.9, 25. , 36.5, 46. , 36.2],
       [35. , 37.3, 27. , 35.5, 36. , 37.5],
       [33. , 35.5, 41. , 36.9, 42. , 36.7],
       [38. , 36.3, 14. , 35.2, 22. , 37.1]])

In [55]:
# Creamos el df a partir de los datos y los índices
health_data = pd.DataFrame(data, index=index, columns=columns)

In [56]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.9  25.0  36.5  46.0  36.2
     2      35.0  37.3  27.0  35.5  36.0  37.5
2014 1      33.0  35.5  41.0  36.9  42.0  36.7
     2      38.0  36.3  14.0  35.2  22.0  37.1

In [57]:
health_data['Bob']

type          HR  Temp
year visit            
2013 1      39.0  37.9
     2      35.0  37.3
2014 1      33.0  35.5
     2      38.0  36.3

In [58]:
health_data['Bob'].unstack()['HR']

visit,1,2
year,,
2013,39.0,35.0
2014,33.0,38.0


In [59]:
health_data['Bob'].unstack()['HR'][1]

year
2013    39.0
2014    33.0
Name: 1, dtype: float64

In [60]:
health_data['Bob']

type          HR  Temp
year visit            
2013 1      39.0  37.9
     2      35.0  37.3
2014 1      33.0  35.5
     2      38.0  36.3

In [61]:
health_data['Bob'].loc[2013]

type,HR,Temp
visit,,
1,39.0,37.9
2,35.0,37.3


### Indexing and Slicing a MultiIndex 

#### Multiply Indexed Series


In [62]:
pop

state       year
California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [63]:
# seleccionamos un valor concreto
pop['California', 2010]


45434534

In [64]:
# partial indexing
pop['Texas']

year
2000    2323647
2010    8984398
dtype: int64

In [65]:
# slicing
pop.loc['New York':'Texas']

state     year
New York  2000     328923
          2010     548954
Texas     2000    2323647
          2010    8984398
dtype: int64

In [66]:
# indexing
pop[:, 2010]

state
California    45434534
New York        548954
Texas          8984398
dtype: int64

In [67]:
# filtering
pop[pop > 1000000]

state       year
California  2000    34343223
            2010    45434534
Texas       2000     2323647
            2010     8984398
dtype: int64

In [68]:
# fancy indexing
pop[['Texas', 'New York']]

state     year
New York  2000     328923
          2010     548954
Texas     2000    2323647
          2010    8984398
dtype: int64

#### Multiply Indexed DataFrames


In [69]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.9  25.0  36.5  46.0  36.2
     2      35.0  37.3  27.0  35.5  36.0  37.5
2014 1      33.0  35.5  41.0  36.9  42.0  36.7
     2      38.0  36.3  14.0  35.2  22.0  37.1

In [70]:
# seleccionamos las columnas
health_data['Sue', 'Temp']

year  visit
2013  1        36.2
      2        37.5
2014  1        36.7
      2        37.1
Name: (Sue, Temp), dtype: float64

In [71]:
# indexing implícito
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      39.0  37.9
     2      35.0  37.3

In [72]:
# indexing implícito (2 primeras filas y 2 primeras columnas)
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      39.0  37.9
     2      35.0  37.3

In [73]:
# indexing explícito (todas las filas y columnas específicas)
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        39.0
      2        35.0
2014  1        33.0
      2        38.0
Name: (Bob, HR), dtype: float64

In [74]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.9  25.0  36.5  46.0  36.2
     2      35.0  37.3  27.0  35.5  36.0  37.5
2014 1      33.0  35.5  41.0  36.9  42.0  36.7
     2      38.0  36.3  14.0  35.2  22.0  37.1

In [75]:
# a través del uso de IndexSlice
idx = pd.IndexSlice

In [76]:
health_data.loc[idx[:,1], idx[:,'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,39.0,25.0,46.0
2014,1,33.0,41.0,42.0


### Rearranging Multi-Indices

Una de las claves de trabajar con multiindices es saber como transformar los datos. Hay una serie de operaciones que preserva la información del dataset, pero lo reordena con el propósito de realizar otra serie de cálculos.

Entre ellas stack() y unstack(), pero existen otras que permiten controlar la reordenación de los datos entre índices y columnas.

#### Sorted and unsorted indices

La mayoría de las operaciones de Slicing sobre multi-indices fallará si estos no están ordenados.

In [100]:
# creamos un indice 
index = pd.MultiIndex.from_product([['a','c', 'b'], [1,2]],
                         names= ['char', 'int'])

In [101]:
data = pd.Series(np.random.rand(6), index= index)

In [102]:
data

char  int
a     1      0.739651
      2      0.753255
c     1      0.641784
      2      0.536724
b     1      0.488613
      2      0.001062
dtype: float64

In [103]:
data['a':'b']

UnsortedIndexError: 'Key length (1) was greater than MultiIndex lexsort depth (0)'

Para evitar esta serie de problemas y poder ordenar los índices, Pandas pone a nuestra disposición dos métodos sort_index() y sortlevel() que nos permite realizar ordenaciones.

In [105]:
data.index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2]],
           labels=[[0, 0, 2, 2, 1, 1], [0, 1, 0, 1, 0, 1]],
           names=['char', 'int'])

In [109]:
# ordenamos los índices
data = data.sort_index()

In [110]:
data.index

MultiIndex(levels=[['a', 'b', 'c'], [1, 2]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=['char', 'int'])

In [112]:
data['a':'b']

char  int
a     1      0.739651
      2      0.753255
b     1      0.488613
      2      0.001062
dtype: float64

#### Stacking and Unstacking indices

Es posible convertir un dataset con un multiindice a una simple representación bidimensional, especificando opcionalmente el level a usar.

Si hacemos un unstack() sobre una Serie convertimos esta en un DataFrame. Para volver a la serie original podemos realizar la operación contraria stack().


In [118]:
pop

state       year
California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [119]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,34343223,328923,2323647
2010,45434534,548954,8984398


In [120]:
pop.unstack(level=1) #por defecto

year,2000,2010
state,,
California,34343223,45434534
New York,328923,548954
Texas,2323647,8984398


In [121]:
pop.unstack() #por defecto

year,2000,2010
state,,
California,34343223,45434534
New York,328923,548954
Texas,2323647,8984398


Lo opuesto a unstack() es stack() que recupera la Serie original

In [123]:
pop.unstack().stack()

state       year
California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

#### Index setting and resetting

Otra forma de reordenar indices jerárquicos es convertir estos en columnas y generar un nuevo índice. Para ello, contamos con el método reset_index().

Si realizamos esto sobre una Serie obtendremos un DataFrame donde los índices se convierten en columnas.

In [126]:
pop

state       year
California  2000    34343223
            2010    45434534
New York    2000      328923
            2010      548954
Texas       2000     2323647
            2010     8984398
dtype: int64

In [124]:
pop_flat = pop.reset_index() # generamos un nuevo índice

In [125]:
pop_flat

,state,year,0
0,California,2000,34343223
1,California,2010,45434534
2,New York,2000,328923
3,New York,2010,548954
4,Texas,2000,2323647
5,Texas,2010,8984398


In [127]:
# utilizamos el argumento name para dar nombre a la columna de población
pop_flat = pop.reset_index(name='population') 

In [128]:
pop_flat

,state,year,population
0,California,2000,34343223
1,California,2010,45434534
2,New York,2000,328923
3,New York,2010,548954
4,Texas,2000,2323647
5,Texas,2010,8984398


A menudo, cuando se está trabajando con datos, el formato de los datasets se parece más a este tipo de datos (pop_flat) y nos es útil construir un multi-indice con algunas de las columnas. Para ello, usamos el método set_index()

In [129]:
pop_flat = pop_flat.set_index(['state', 'year'])

In [130]:
pop_flat

population
state      year            
California 2000    34343223
           2010    45434534
New York   2000      328923
           2010      548954
Texas      2000     2323647
           2010     8984398

### Data Aggregations on Multi-Indices

Podemos realizar operaciones típicas de agrupación del tipo mean(), sum(), max(),... eligiendo el nivel del índice para la agrupación. Para ello usamos el argumento "level" sobre la agrupación.

In [131]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.9  25.0  36.5  46.0  36.2
     2      35.0  37.3  27.0  35.5  36.0  37.5
2014 1      33.0  35.5  41.0  36.9  42.0  36.7
     2      38.0  36.3  14.0  35.2  22.0  37.1

In [132]:
health_data.mean()

subject  type
Bob      HR      36.250
         Temp    36.750
Guido    HR      26.750
         Temp    36.025
Sue      HR      36.500
         Temp    36.875
dtype: float64

In [133]:
health_data.mean(level='year')

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
year                                         
2013     37.0  37.6  26.0  36.00  41.0  36.85
2014     35.5  35.9  27.5  36.05  32.0  36.90

In [136]:
health_data.mean(level='visit')

subject   Bob       Guido          Sue       
type       HR  Temp    HR   Temp    HR   Temp
visit                                        
1        36.0  36.7  33.0  36.70  44.0  36.45
2        36.5  36.8  20.5  35.35  29.0  37.30

In [142]:
health_data.sum(level='visit')

subject   Bob       Guido         Sue      
type       HR  Temp    HR  Temp    HR  Temp
visit                                      
1        72.0  73.4  66.0  73.4  88.0  72.9
2        73.0  73.6  41.0  70.7  58.0  74.6

Además, utilizando el argumento "axis" podemos elegir también entre los niveles de las columnas

In [139]:
health_data.index

MultiIndex(levels=[[2013, 2014], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]],
           names=['year', 'visit'])

In [140]:
health_data.columns

MultiIndex(levels=[['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]],
           names=['subject', 'type'])

In [144]:
health_data.mean(axis= 1, level='type')

type               HR       Temp
year visit                      
2013 1      36.666667  36.866667
     2      32.666667  36.766667
2014 1      38.666667  36.366667
     2      24.666667  36.200000

In [145]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      39.0  37.9  25.0  36.5  46.0  36.2
     2      35.0  37.3  27.0  35.5  36.0  37.5
2014 1      33.0  35.5  41.0  36.9  42.0  36.7
     2      38.0  36.3  14.0  35.2  22.0  37.1